In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# WIP: Decoding

In [ ]:
import numpy as np

npz_fname = 'data/look/epoch-20231214-filtered-trainval.npz'
# npz_fname = 'data/look/epoch20240221_expanded10x_trainval.npz'

ds = np.load(npz_fname, encoding='latin1', allow_pickle=True)
# print(f"num drawings in dataset: {ds.shape[0]}")

train = ds['train']
valid = ds['valid']

len(train), max([len(train[i]) for i in range(len(train))])

(1800, 300)

In [ ]:
import torch
from singleline_models.sketch_transformer.trainer import get_default_config, Trainer

run_id = 'rb1irlmk'

t2 = Trainer.load(run_dir=f'models/{run_id}', run_id=run_id, device='cpu')

t2.model

RUN_ID: 0113464

HYPERPARAMETERS:

{
  "n_layer": 4,
  "n_head": 8,
  "d_model": 128,
  "d_ff": 512,
  "d_lowerdim": 256,
  "vocab_size": null,
  "block_size": null,
  "dropout_rate": 0.1,
  "max_seq_length": 250,
  "batch_size": 100,
  "blind_decoder_mask": true,
  "dataset_source": "look",
  "dataset_name": "look_i16__minn10_epsilon1",
  "dataset_fname": "data/look/epoch20240221_expanded10x_trainval.npz",
  "augment_stroke_prob": 0.1,
  "use_random_scale": true,
  "random_scale_factor": 0.15,
  "epochs": 50000,
  "lr": 0.001,
  "use_lr_decay": true,
  "min_lr": 1e-05,
  "lr_decay": 0.9999
}


finished filtering - len(dataset) = 30800, len(data) = 30800
finished filtering - len(dataset) = 3300, len(data) = 3300


Model(
  (encoder): Encoder(
    (embedding): Linear(in_features=5, out_features=128, bias=True)
    (pos_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (enc_layers): ModuleList(
      (0-3): 4 x EncoderLayer(
        (mha): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (ffn): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
        (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (bottleneck_layer): SelfAttn(
    (embedding_layer): Linear(in_features=128, out_features=256, bias=True)
  )
  (expand_layer): DenseExpander(
    (project_layer): Linear(in_features=256, out_features=128, 

In [ ]:
hp = trainer.hp

hp.__dict__

{'n_layer': 4,
 'n_head': 8,
 'd_model': 128,
 'd_ff': 512,
 'd_lowerdim': 256,
 'vocab_size': None,
 'block_size': None,
 'dropout_rate': 0.1,
 'max_seq_length': 250,
 'batch_size': 100,
 'blind_decoder_mask': True,
 'dataset_source': 'look',
 'dataset_name': 'look_i16__minn10_epsilon1',
 'dataset_fname': 'data/look/epoch-20231214-filtered-trainval.npz',
 'augment_stroke_prob': 0.1,
 'use_random_scale': True,
 'random_scale_factor': 0.15,
 'epochs': 50000,
 'lr': 0.001,
 'use_lr_decay': True,
 'min_lr': 1e-05,
 'lr_decay': 0.9999,
 'learning_rate': 0.001}

In [ ]:
valid_iter = iter(t2.valid_loader)

data, mask = next(valid_iter)

data.shape, mask.shape

(torch.Size([100, 252, 5]), torch.Size([100, 251]))

In [ ]:
data[0]

tensor([[ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
        [ 3.3485,  8.8307,  1.0000,  0.0000,  0.0000],
        [ 0.0000, -0.4527,  1.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000]])

In [ ]:
from singleline_dataset.transforms import *
from singleline_dataset.display import *

In [ ]:
data.shape

torch.Size([100, 252, 5])

In [ ]:
# # back to stroke-3
# data[..., [0,1,3]]

In [ ]:
from singleline_models.sketch_transformer.masks import *

In [ ]:
t2.model.eval()

with torch.no_grad():
    inp = data
    tar_inp = inp[:, :-1, ...]
    tar_real = inp[:, 1:, ...]

    enc_padding_mask, dec_padding_mask, dec_target_padding_mask, look_ahead_mask = create_masks(inp, tar_inp, device=t2.device)

    recon, _ = t2.model(inp, tar_inp, enc_padding_mask, dec_padding_mask, dec_target_padding_mask, look_ahead_mask)


In [ ]:
recon.shape

torch.Size([100, 251, 5])

In [ ]:
recon

tensor([[[ 3.5329e+00,  9.6288e+00,  5.3611e+00, -3.4596e+00, -8.3489e+00],
         [ 2.3294e-02, -3.8949e-01,  9.8829e+00, -8.5926e+00, -1.2378e+01],
         [ 1.1838e-01, -7.8708e-02,  1.0081e+01, -8.7456e+00, -1.2712e+01],
         ...,
         [-3.1393e-02, -9.3986e-01, -3.4550e+00,  7.3045e+00, -1.3964e+01],
         [-1.9621e-01, -1.6634e-01,  5.2449e-01,  2.7607e+00, -1.5029e+01],
         [-8.3308e-02, -7.3363e-01,  5.8782e-02,  3.1838e+00, -1.4897e+01]],

        [[ 4.9594e+00,  1.0663e+01,  5.3578e+00, -3.5522e+00, -7.5788e+00],
         [ 3.0547e-03, -3.5476e-01,  1.0119e+01, -8.8608e+00, -1.2451e+01],
         [ 1.2438e-01, -1.3330e-02,  1.0128e+01, -8.8334e+00, -1.2655e+01],
         ...,
         [-2.1159e-01, -1.0479e+00, -2.5831e+00,  6.2649e+00, -1.3984e+01],
         [ 3.5933e-02, -2.9249e-01,  4.2993e-01,  2.7182e+00, -1.4733e+01],
         [ 1.1287e-01, -8.7293e-01,  1.0823e-01,  3.1185e+00, -1.4944e+01]],

        [[ 2.3355e+00,  1.1197e+01,  5.5395e+00, -3.5123

In [ ]:
emb, enc_output = t2.model.encode_from_seq(data)

In [ ]:
emb.shape

torch.Size([100, 256])

In [ ]:
from torch import tensor

decoder_input = tensor([0., 0., 1., 0., 0.])
output = torch.ones((emb.shape[0], 1, 5)) * decoder_input

output.shape

torch.Size([100, 1, 5])

In [ ]:
output

tensor([[[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0.]],

        [[

In [ ]:

i = 1
expected_len = hp.max_seq_length + 2

# for i in range(seq_len):
nattn = expected_len if expected_len is not None else i + 1

enc_input_dummy = make_dummy_input(
    total_seq_len=expected_len, nattn=nattn, batch_size=emb.shape[0])


In [ ]:
# enc_dummy_input

In [ ]:
enc_padding_mask, dec_padding_mask, dec_target_padding_mask, look_ahead_mask = \
    create_masks(enc_input_dummy, output, device='cpu')


In [ ]:
# dec_target_padding_mask

In [ ]:
final_output, attention_weights = t2.model.decode(
            embedding=emb,
            target=output,
            dec_padding_mask=dec_padding_mask,
            dec_target_padding_mask=dec_target_padding_mask,
            look_ahead_mask=look_ahead_mask)

In [ ]:
final_output.shape

torch.Size([100, 2, 5])

In [ ]:
final_output[1]

tensor([[-1.2349e-01, -3.6243e-02,  9.7793e+00, -8.2115e+00, -1.3756e+01],
        [-7.4654e-02,  9.4675e-03,  9.7838e+00, -8.1558e+00, -1.4080e+01]],
       grad_fn=<SelectBackward0>)

In [ ]:
final_output[2]

tensor([[-1.2319e-01, -3.6380e-02,  9.7818e+00, -8.2146e+00, -1.3754e+01],
        [-7.4559e-02,  9.5914e-03,  9.7832e+00, -8.1550e+00, -1.4080e+01]],
       grad_fn=<SelectBackward0>)

In [ ]:
predictions = final_output[:, -1:, ...]  # (batch_size, 1, vocab_size)

In [ ]:
torch.softmax(predictions[..., 2:], dim=-1).argmax(dim=-1, keepdim=True).shape

torch.Size([100, 1, 1])

In [ ]:
from torch.nn import functional as F

pred_location = predictions[..., :2]

pred_metadata = F.one_hot(predictions[..., 2:].argmax(dim=-1), num_classes=3)

predicted = torch.cat([pred_location, pred_metadata], dim=-1)

predicted.shape

torch.Size([100, 1, 5])

In [ ]:
predicted[0]

tensor([[-0.0746,  0.0096,  1.0000,  0.0000,  0.0000]],
       grad_fn=<SelectBackward0>)

In [ ]:
output = torch.cat([output, predicted], dim=1)

In [ ]:
output.shape

torch.Size([100, 3, 5])

In [ ]:
output[0]

tensor([[ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
        [-0.0506, -0.0034,  1.0000,  0.0000,  0.0000],
        [-0.0746,  0.0096,  1.0000,  0.0000,  0.0000]],
       grad_fn=<SelectBackward0>)

In [ ]:
output[1]

tensor([[ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
        [-0.0506, -0.0034,  1.0000,  0.0000,  0.0000],
        [-0.0747,  0.0095,  1.0000,  0.0000,  0.0000]],
       grad_fn=<SelectBackward0>)

In [ ]:
# predicted = predictions
# predicted = tf.concat((predicted[..., :2],
#                        tf.nn.softmax(predicted[..., 2:], axis=-1)), axis=-1)
# finished_ones = np.sum(tf.argmax(predicted[...,  2:], axis=-1) == 2)
# output = tf.concat([output, predicted], axis=1)
# if finished_ones == emb.shape[0]:
#     break

In [ ]:
# from singleline_stroke3.display import *
# from singleline_stroke3.transforms import *